In [1]:
!pip install azure-storage-blob

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from azure.storage.blob import BlobServiceClient

In [3]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import StringIO


# Your Azure Storage connection details
config_file_path = "config.json"

with open(config_file_path,"r") as config_file:
    config = json.load(config_file)

connection_string = config["connectionString"]
container_name = "violation"

# Initialize the Azure Blob service client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Base URL for your data source
base_url = "https://data.cityofnewyork.us/resource/nc67-uf89.csv"

# Parameters for API request and batch processing
limit = 1000000
total_records = 111625885
offset = 63000000  # Start from the beginning if you aim to download the entire dataset

# Dataframe to accumulate records
accumulated_df = pd.DataFrame()

while offset < total_records:
    # Update the URL with the current offset
    data_url = f"{base_url}?$limit={limit}&$offset={offset}"

    try:
        # Download the current chunk of data
        df_chunk = pd.read_csv(data_url)

        # Accumulate the data
        accumulated_df = pd.concat([accumulated_df, df_chunk], ignore_index=True)

        # Check if accumulated data reached the batch size of 5,000,000 records or if it's the last chunk
        if len(accumulated_df) >= 5000000 or offset + limit >= total_records:
            # Convert the accumulated DataFrame to a CSV string
            csv_data = StringIO()
            accumulated_df.to_csv(csv_data, index=False)
            csv_data.seek(0)

            # Define the blob name in your container
            blob_name = f"data_chunk_offset_{offset}.csv"

            # Upload the batch to Azure Blob Storage
            blob_client = container_client.get_blob_client(blob_name)
            blob_client.upload_blob(csv_data.getvalue(), overwrite=True)
            print(f"Successfully uploaded {blob_name}")

            # Reset the accumulated dataframe
            accumulated_df = pd.DataFrame()

    except Exception as e:
        print(f"Error during data fetch/upload: {e}")
        break  # or handle retry logic

    # Increment the offset for the next loop iteration
    offset += limit


Successfully uploaded data_chunk_offset_67000000.csv
Successfully uploaded data_chunk_offset_72000000.csv
Successfully uploaded data_chunk_offset_77000000.csv
Successfully uploaded data_chunk_offset_82000000.csv
Successfully uploaded data_chunk_offset_87000000.csv
Successfully uploaded data_chunk_offset_92000000.csv
Successfully uploaded data_chunk_offset_97000000.csv
Successfully uploaded data_chunk_offset_102000000.csv
Successfully uploaded data_chunk_offset_107000000.csv
Successfully uploaded data_chunk_offset_111000000.csv
